In [1]:
import pandas as pd
df = pd.read_csv('C001_FakeHypotension.csv')
print(df.head())

   Unnamed: 0        MAP  diastolic_bp  systolic_bp      urine        ALT  \
0           0  67.964940     56.709198   144.578430  241.00052  24.226444   
1           1  63.603493     53.635162   143.283780  230.52171  23.933348   
2           2  65.750670     56.904236   143.534000  217.76741  21.803823   
3           3  44.684580     41.952940   117.236534  291.90466  33.613720   
4           4  61.486233     63.960106   111.537060  435.26230  45.407272   

         AST        PO2  lactic_acid  serum_creatinine  ...  GCS_total  \
0  23.811718   83.46306     1.477299          0.893153  ...         15   
1  30.188170  120.01681     1.733099          0.862924  ...         15   
2  26.318861   84.16420     1.375228          0.819521  ...         15   
3  24.136852  102.35325     1.252799          0.754315  ...         15   
4  23.192320   63.73771     1.845126          1.155209  ...         15   

   urine_m  ALT_AST_m  FiO2_m  GCS_total_m  PO2_m  lactic_acid_m  \
0        0          0   

In [ ]:
#States (of the patient):
#MAP: Mean Arterial Pressure. The average pressure in a patient's arteries during one cardiac cycle. It's a key indicator of tissue perfusion in organs.
#normal range: 70-100 mmHg
#severe risk: below 60

#Diastolic BP: Diastolic Blood Pressure. The pressure in the arteries when the heart rests between beats.
#normal range: 60-80 mmHg
#severe risk: below 40 or above 120

#Systolic BP: Systolic Blood Pressure. The pressure in the arteries when the heart beats and pumps blood.
#normal range: 90-120 mmHg
#severe risk: below 90 or above 180

#Urine: Volume of urine output, an important measure of kidney function and fluid balance.
#Urine_m: Indicates whether urine measurement is missing.
#normal range: 33-83mL per hour (not sure what the time interval is??)

#ALT: Alanine Aminotransferase. An enzyme found in the liver, used to assess liver health. Elevated levels indicate liver damage or disease.
#normal range: 7-56 U/L
#severe risk: above 200
#AST: Aspartate Aminotransferase. Another enzyme linked to liver function, often measured with ALT to assess liver damage.
#normal range: 10-40 U/L
#severe risk: above 200
#ALT_AST_m: Indicates whether ALT and AST measurements are missing.


#PO2: Partial Pressure Oxygen. The amount of oxygen gas in arterial blood, used to assess a patient's respitory function.
#PO2_m: Indicates whether PO2 measurement is missing.
#normal range: 75-100 mmHg
#severe risk: below 60

#Lactic Acid: A product of anaerobic metabolism. Elevated levels can indicate inadequate oxygen delivery to tissues or sepsis.
#Lactic_acid_m: Indicates whether lactic acid measurement is missing.
#normal range: 0.5-2.2 mmol/L
#severe risk: above 4

#Serum Creatinine: A waste product filtered by the kidneys. High levels can indicate impaired kidney function.
#Serum_creatinine_m: Indicates whether serum creatinine measurement is missing.
#normal range: 0.6-1.2 mg/dL
#severe risk: above 4

#GCS Total: Glasgow Coma Scale Total. A scale that assesses the level of consciousness in a patient based on their verbal, motor, and eye responses.
#GCS_total_m: Indicates whether GCS score is missing.
#normal range: 15 (fully conscious)
#severe risk: 3-7 (comatose)


#Actions (of the doctor):
#Fluid Boluses: Amount of fluids administered intravenously to manage blood pressure or hydration.

#Vasopressors: Medications that constrict blood vessels to increase blood pressure, often used in hypotensive or shock states.

#FiO2: Fraction of Inspired Oxygen. The percentage of oxygen in the air mixture that a patient is breathing. Higher levels are used in patients needing more oxygen.
#FiO2_m: Indicates whether FiO2 measurements are missing.


In [2]:
#import libraries
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
#combine adjacent records into transition (takes 3min to run, can skip and read saved file)
#each transition combines 2 rows of data with same PatientID, adjacent Timepoints
state_columns = ['MAP', 'diastolic_bp', 'systolic_bp', 'urine', 'ALT', 'AST', 'PO2', 'lactic_acid', 'serum_creatinine', 'GCS_total']
action_columns = ['fluid_boluses', 'vasopressors', 'FiO2']

def create_transitions(df):
    df = df.sort_values(['PatientID', 'Timepoints'])
    transitions = []
    patients = df['PatientID'].unique()

    for patient in patients:
        patient_records = df[df['PatientID'] == patient].reset_index(drop=True)
        for i in range(len(patient_records) - 1):
            current_state = patient_records.iloc[i][state_columns]
            next_state = patient_records.iloc[i + 1][state_columns]
            action = patient_records.iloc[i][action_columns]
            transitions.append(np.concatenate([current_state, action, next_state]))
    return np.array(transitions)

transitions = create_transitions(df)
#print(transitions)

transitions_header = ['current_' + x for x in state_columns] + action_columns + ['next_' + x for x in state_columns]
transitions_df = pd.DataFrame(transitions, columns=transitions_header)
transitions_df.to_csv("processed_transitions.csv")

[[ 67.96494     56.709198   144.57843    ...   1.7330985    0.8629241
   15.        ]
 [ 63.603493    53.635162   143.28378    ...   1.3752282    0.8195206
   15.        ]
 [ 65.75067     56.904236   143.534      ...   1.2527988    0.75431526
   15.        ]
 ...
 [ 78.85624     64.86505    123.241035   ...   1.3828557    1.086653
   15.        ]
 [ 74.18521     58.027935   123.97409    ...   1.2868271    0.9677528
   15.        ]
 [ 88.85414     69.937996   125.38483    ...   1.2924969    0.95900583
   15.        ]]


In [5]:
#read output of above code from file directly
transitions_df = pd.read_csv('processed_transitions.csv')
transitions = transitions_df.to_numpy()

In [6]:
#standardize and split data
scaler = StandardScaler()
scaled_transitions = scaler.fit_transform(transitions)

X = scaled_transitions[:, :len(state_columns) + len(action_columns)]
y = scaled_transitions[:, len(state_columns) + len(action_columns):]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [17]:
#use neural network to functionally approximate transitions
class TransitionModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_size=128):
        super(TransitionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

model = TransitionModel(input_size=len(state_columns) + len(action_columns), output_size=len(state_columns))
loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

batch_size = 16
for epoch in range(100):
    model.train()
    X_train_shuffled_idx = torch.randperm(X_train_tensor.size()[0])
    for i in range(0, X_train_tensor.size()[0], batch_size):
        idx = X_train_shuffled_idx[i:i + batch_size]
        batch_X, batch_y = X_train_tensor[idx], y_train_tensor[idx]
        outputs = model(batch_X)
        loss = loss_function(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f'epoch: {epoch}, loss: {loss.item()}')

model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    test_loss = loss_function(y_pred, y_test_tensor)
    print(f'test loss: {test_loss.item()}')

epoch: 0, loss: 1.6463706493377686
epoch: 10, loss: 0.1986122727394104
epoch: 20, loss: 0.7940933704376221
epoch: 30, loss: 0.2832067608833313
epoch: 40, loss: 0.4175243377685547
epoch: 50, loss: 0.26314106583595276
epoch: 60, loss: 0.540831446647644
epoch: 70, loss: 0.8002191781997681
epoch: 80, loss: 0.2809102237224579
epoch: 90, loss: 0.06554313004016876
test loss: 0.5617451667785645


In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

linear_model = LinearRegression()

linear_model.fit(X_train, y_train)
y_pred_train = linear_model.predict(X_train)
y_pred_test = linear_model.predict(X_test)
train_loss = mean_squared_error(y_train, y_pred_train)
test_loss = mean_squared_error(y_test, y_pred_test)

print(f"Train Loss (Linear Regression): {train_loss}")
print(f"Test Loss (Linear Regression): {test_loss}")

Train Loss (Linear Regression): 0.5522047393013321
Test Loss (Linear Regression): 0.563679192135602


In [14]:
from sklearn.tree import DecisionTreeRegressor

tree_model = DecisionTreeRegressor(max_depth=10, random_state=42)
tree_model.fit(X_train, y_train)

y_pred_train = tree_model.predict(X_train)
y_pred_test = tree_model.predict(X_test)

train_loss_tree = mean_squared_error(y_train, y_pred_train)
test_loss_tree = mean_squared_error(y_test, y_pred_test)

print(f"Train Loss (Decision Tree): {train_loss_tree}")
print(f"Test Loss (Decision Tree): {test_loss_tree}")

Train Loss (Decision Tree): 0.5323839253554765
Test Loss (Decision Tree): 0.5781279933566097


'''
model based approach progress:
1. preprocessing of states & actions
 - currently ignoring the _m columns (indications of missing state measurements)
2. function approximation for transitions
 - tried neural networks, linear regression and decision tree, all have the same loss around 0.5
 - for neural network, the loss didn't improve from initial epoch, need more work on the architecture & parameter values
 - consider using multiple previous states instead of just 1 (like atari games)
 - is it important to consider which transitions are from the same patient?
 - consider mapping the values to categorical values? (severely low, moderately low, normal, moderately high, severely high)
3. design rewards
 - listed normal range and severe risk range for each state measurement
 - are some of the measurements more significant than others (eg GCS - measurement of consciousness)?
 - do we need a terminal reward? is the transition's position in the sequence (eg at the end) important to the reward value?
4. extract policy from existing data
 - not started
5. calculate optimal policy with doctor's policy as starting point
 - not started
6. evaluate policy
 - not started
'''